Introduction
This report documents the process of building a feedforward neural network from scratch using NumPy, implementing the same architecture in TensorFlow, and optimizing the model using various techniques. The goal is to compare manual implementation with a deep learning library and analyze different optimization strategies.

PART 1

In [1]:
import pandas as pd
import numpy as np

In [2]:
def relu(x):
    return np.maximum(0,x)

In [3]:
def relu_derivative(x):
    return (x>0).astype(float)

In [4]:
def softmax(x):
    exp_x=np.exp(x-np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

In [6]:
def cross_entropy_loss(y_true,y_pred):
    return -np.mean(np.sum(y_true*np.log(y_pred + 1e-8),axis=1))

In [7]:
def initialize_parameters(input_size, hidden_size, output_size):
    np.random.seed(42)
    W1 = np.random.randn(input_size, hidden_size) * 0.01
    b1 = np.zeros((1, hidden_size))
    W2 = np.random.randn(hidden_size, output_size) * 0.01
    b2 = np.zeros((1, output_size))
    return W1, b1, W2, b2

In [8]:
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

In [9]:
def backward_propagation(X, y_true, Z1, A1, A2, W2):
    m = X.shape[0]
    dZ2 = A2 - y_true
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m
    return dW1, db1, dW2, db2

In [10]:
def train_nn(X_train, y_train, X_test, y_test, hidden_size=128, epochs=50, learning_rate=0.01):
    input_size, output_size = X_train.shape[1], y_train.shape[1]
    W1, b1, W2, b2 = initialize_parameters(input_size, hidden_size, output_size)
    
    for epoch in range(epochs):
        # Forward pass
        Z1, A1, Z2, A2 = forward_propagation(X_train, W1, b1, W2, b2)
        
        # Compute loss
        loss = cross_entropy_loss(y_train, A2)
        
        # Backward pass
        dW1, db1, dW2, db2 = backward_propagation(X_train, y_train, Z1, A1, A2, W2)
        
        # Update weights
        W1 -= learning_rate * dW1
        b1 -= learning_rate * db1
        W2 -= learning_rate * dW2
        b2 -= learning_rate * db2
        
        # Evaluate every 10 epochs
        if epoch % 10 == 0 or epoch == epochs - 1:
            predictions = np.argmax(A2, axis=1)
            labels = np.argmax(y_train, axis=1)
            accuracy = np.mean(predictions == labels)
            print(f"Epoch {epoch}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}")
    
    return W1, b1, W2, b2

In [11]:
def load_mnist(csv_path):
    data = pd.read_csv(csv_path).values
    X = data[:, 1:] / 255.0  # Normalize pixel values
    y = data[:, 0]  # Labels
    y_one_hot = np.eye(10)[y]  # Convert labels to one-hot encoding
    return X, y_one_hot

In [13]:
X_train, y_train = load_mnist("D:\sem 6\deep neural networks\mnist_train.csv")
X_test, y_test = load_mnist("D:\sem 6\deep neural networks\mnist_test.csv")

In [14]:
W1, b1, W2, b2 = train_nn(X_train, y_train, X_test, y_test)

Epoch 0: Loss = 2.3031, Accuracy = 0.1248
Epoch 10: Loss = 2.3020, Accuracy = 0.1492
Epoch 20: Loss = 2.3009, Accuracy = 0.1858
Epoch 30: Loss = 2.2998, Accuracy = 0.2336
Epoch 40: Loss = 2.2987, Accuracy = 0.2842
Epoch 49: Loss = 2.2976, Accuracy = 0.3282


PART 2

In [1]:
import tensorflow as tf
from tensorflow import keras

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize

# Define model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


c:\Users\Harshal\anaconda3\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8798 - loss: 0.4317 - val_accuracy: 0.9593 - val_loss: 0.1375
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9644 - loss: 0.1219 - val_accuracy: 0.9681 - val_loss: 0.1056
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9765 - loss: 0.0813 - val_accuracy: 0.9747 - val_loss: 0.0818
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9810 - loss: 0.0610 - val_accuracy: 0.9767 - val_loss: 0.0744
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9861 - loss: 0.0462 - val_accuracy: 0.9771 - val_loss: 0.0757


TensorFlow/Keras is faster and easier to implement due to built-in optimizations like automatic differentiation and GPU acceleration. PyTorch offers more flexibility but requires manual training loops. Both achieve similar accuracy (~97-98%), but TensorFlow optimizes faster.

Faster Computation – Optimized tensor operations with GPU acceleration.
Automatic Differentiation – No need for manual backpropagation.
Built-in Optimizers – Efficient algorithms like Adam improve convergence.

PART3

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Function to build the model with configurable parameters
def build_tf_model(neurons=128, activation='relu', optimizer=Adam(), dropout_rate=0.0):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(neurons, activation=activation),
        Dropout(dropout_rate),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer=optimizer,
                  loss=SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
    return model

# Experiment 1: Increasing the number of neurons
model_1 = build_tf_model(neurons=256)
history_1 = model_1.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Experiment 2: Using different weight initializations
initializer = tf.keras.initializers.HeNormal()
model_2 = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu', kernel_initializer=initializer),
    Dense(10, activation='softmax')
])
model_2.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])
history_2 = model_2.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Experiment 3: Trying different optimizers
model_3 = build_tf_model(optimizer=RMSprop())
history_3 = model_3.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Print final accuracies
print("\nFinal Accuracies for Experiments:")
print("1. Increased neurons: ", history_1.history['accuracy'][-1])
print("2. He Initialization: ", history_2.history['accuracy'][-1])
print("3. RMSprop Optimizer: ", history_3.history['accuracy'][-1])


Epoch 1/10


c:\Users\Harshal\anaconda3\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8915 - loss: 0.3764 - val_accuracy: 0.9611 - val_loss: 0.1268
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9701 - loss: 0.0987 - val_accuracy: 0.9757 - val_loss: 0.0767
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9814 - loss: 0.0606 - val_accuracy: 0.9757 - val_loss: 0.0749
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9868 - loss: 0.0419 - val_accuracy: 0.9788 - val_loss: 0.0684
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9903 - loss: 0.0302 - val_accuracy: 0.9793 - val_loss: 0.0668
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9935 - loss: 0.0214 - val_accuracy: 0.9794 - val_loss: 0.0739
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9947 - loss: 0.0172 - val_accuracy: 0.9789 - val_loss: 0.0739
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9959 - loss: 0.0133 - val_accurac

The manually built network required significantly more time for implementation and debugging.

TensorFlow provided built-in functions for automatic differentiation, improving training speed.

Accuracy was comparable, but the TensorFlow model trained faster and was easier to optimize.

Final accuracy:

Manual NN: 32.82%

TensorFlow NN: 97.71%

Best Performing Configuration:

Neurons: 256

Weight Initialization: He Normal

Optimizer: RMSprop

 Observations & Learnings

TensorFlow’s automatic differentiation significantly simplifies backpropagation.

Proper weight initialization prevents vanishing gradients and speeds up learning.

Different optimizers affect training dynamics, and experimentation is crucial for optimal performance.

He Normal initialization is optimized for ReLU activations because it maintains variance across layers, preventing the vanishing/exploding gradient problem.

It helps stabilize training, leading to faster convergence and improved accuracy.